# 🏍️ Ithaka Motorcycle Powertrain Designer

Welcome to the **Ithaka Motorcycle Powertrain Design & Simulation Tool**. This notebook provides a comprehensive, no-code interface for:

- 🏍️ **Selecting from predefined motorcycles** or building custom configurations
- 🗺️ **Testing on real-world GPS tracks** to simulate performance
- 🚀 **Running physics-based simulations** with a single click
- 📊 **Viewing detailed results** with professional visualizations
- 💾 **Exporting comprehensive data** for engineering analysis

**No coding required** - simply run each cell in order and use the interactive interfaces that appear.

In [ ]:
#@title 🚀 Environment Setup (Click to expand if needed) { display-mode: "form" }

#@markdown This cell automatically detects your environment and sets up the required dependencies.
#@markdown **No action needed** - just run this cell and wait for the ✅ success message.

import sys
import os
import warnings
import subprocess
warnings.filterwarnings('ignore')

def setup_environment():
    """Smart environment setup that works in both Colab and local environments."""
    
    # Detect if we're in Google Colab
    try:
        import google.colab
        IN_COLAB = True
        print("📦 Google Colab environment detected")
    except ImportError:
        IN_COLAB = False
        print("💻 Local environment detected")
    
    if IN_COLAB:
        # Smart repository detection and setup for Colab
        print("🔍 Checking for existing repository...")
        
        # Check if we're already in a valid repository
        current_dir = os.getcwd()
        possible_repo_paths = [
            '/content/Teo-Moto-Repo',
            '/content/ithaka-powertrain-sim', 
            current_dir
        ]
        
        repo_found = False
        repo_path = None
        for path in possible_repo_paths:
            if os.path.exists(path):
                # Check if it contains our package
                setup_py_path = os.path.join(path, 'setup.py')
                if os.path.exists(setup_py_path):
                    print(f"✅ Found existing repository at: {path}")
                    os.chdir(path)
                    repo_found = True
                    repo_path = path
                    break
        
        if not repo_found:
            print("📥 Cloning repository from GitHub...")
            
            # Try to detect the correct repository URL
            repo_urls = [
                'https://github.com/Teodus/Teo-Moto-Repo.git',
                'https://github.com/Teodus/ithaka-powertrain-sim.git'
            ]
            
            clone_success = False
            for repo_url in repo_urls:
                try:
                    print(f"   Trying: {repo_url}")
                    result = subprocess.run(['git', 'clone', repo_url], 
                                          capture_output=True, text=True, cwd='/content')
                    if result.returncode == 0:  # Success
                        # Extract repo name from URL
                        repo_name = repo_url.split('/')[-1].replace('.git', '')
                        repo_path = f'/content/{repo_name}'
                        
                        if os.path.exists(repo_path):
                            print(f"✅ Successfully cloned to: {repo_path}")
                            os.chdir(repo_path)
                            clone_success = True
                            break
                except Exception as e:
                    print(f"   ❌ Failed: {e}")
                    continue
            
            if not clone_success:
                print("❌ Could not clone repository. Please check your internet connection.")
                print("💡 Manual fix: Run '!git clone https://github.com/Teodus/Teo-Moto-Repo.git' in a new cell")
                return False
        
        # Install requirements first, then package
        print("📦 Installing requirements...")
        if os.path.exists('requirements.txt'):
            try:
                result = subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], 
                                      capture_output=True, text=True, timeout=300)
                if result.returncode == 0:
                    print("✅ Requirements installed successfully")
                else:
                    print(f"⚠️ Requirements installation warnings: {result.stderr}")
            except Exception as e:
                print(f"⚠️ Requirements installation issue: {e}")
        
        print("📦 Installing package in development mode...")
        try:
            result = subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.'], 
                                  capture_output=True, text=True, timeout=180)
            if result.returncode == 0:
                print("✅ Package installed successfully")
            else:
                print(f"⚠️ Package installation warnings: {result.stderr}")
        except Exception as e:
            print(f"⚠️ Package installation issue: {e}")
    
    else:
        print("✅ Local environment - assuming package is already available")
    
    # Verify installation by importing key modules
    try:
        print("🔍 Verifying package installation...")
        
        # Test core imports first
        import pandas as pd
        import numpy as np
        import matplotlib.pyplot as plt
        print("✅ Core scientific libraries available")
        
        # Test Jupyter widgets
        import ipywidgets as widgets
        from IPython.display import display, clear_output, HTML
        print("✅ Jupyter widgets available")
        
        # Test other dependencies
        import json
        import glob
        from datetime import datetime
        print("✅ Standard libraries available")
        
        # Test project imports
        from ithaka_powertrain_sim.component_library import (
            get_motorcycle_list, get_motorcycle_specs, create_motorcycle,
            ComponentPickerState, PREDEFINED_MOTORCYCLES
        )
        from ithaka_powertrain_sim.motorbike import Motorbike
        from ithaka_powertrain_sim.trajectory import load_gpx, append_and_resample_dataframe
        print("✅ Project modules imported successfully")
        
        print(f"📊 Found {len(get_motorcycle_list())} predefined motorcycles")
        
        # Find GPX files
        gpx_files = glob.glob('docs/gpx_files/*.gpx')
        if gpx_files:
            print(f"🗺️ Found {len(gpx_files)} test tracks")
        else:
            print("⚠️ No GPX files found - track simulation may be limited")
        
        return True
        
    except ImportError as e:
        print(f"❌ Import failed: {e}")
        if 'shortuuid' in str(e):
            print("💡 Installing missing shortuuid dependency...")
            try:
                subprocess.run([sys.executable, '-m', 'pip', 'install', 'shortuuid'], 
                             capture_output=True, text=True, timeout=60)
                print("✅ shortuuid installed, please re-run this cell")
            except Exception as install_err:
                print(f"❌ Failed to install shortuuid: {install_err}")
        print("💡 Try restarting the runtime and running this cell again")
        return False
    except Exception as e:
        print(f"❌ Package verification failed: {e}")
        print("💡 Try restarting the runtime and running this cell again")
        return False

# Run the setup
setup_success = setup_environment()

if setup_success:
    print()
    print("🎉 Setup complete! Ready to design motorcycles!")
    print("👇 Continue to Step 1 below")
else:
    print()
    print("❌ Setup failed. Please check the error messages above.")

In [ ]:
#@title 🚀 Environment Setup { display-mode: "form" }

#@markdown This cell automatically detects your environment and sets up all required dependencies.
#@markdown **Click the play button** to run this cell - setup takes about 30-60 seconds.

import sys
import os
import warnings
import subprocess
warnings.filterwarnings('ignore')

def setup_environment():
    """Smart environment setup for both Colab and local environments."""
    
    # Detect environment
    try:
        import google.colab
        IN_COLAB = True
        print("📦 Google Colab environment detected")
    except ImportError:
        IN_COLAB = False
        print("💻 Local environment detected")
    
    if IN_COLAB:
        # Check for existing repository
        print("🔍 Checking for repository...")
        
        current_dir = os.getcwd()
        possible_paths = ['/content/Teo-Moto-Repo', '/content/ithaka-powertrain-sim', current_dir]
        
        repo_found = False
        for path in possible_paths:
            if os.path.exists(path) and os.path.exists(os.path.join(path, 'setup.py')):
                print(f"✅ Found repository at: {path}")
                os.chdir(path)
                repo_found = True
                break
        
        if not repo_found:
            print("📥 Cloning repository...")
            try:
                result = subprocess.run(['git', 'clone', 'https://github.com/Teodus/Teo-Moto-Repo.git'], 
                                      capture_output=True, text=True, cwd='/content')
                if result.returncode == 0:
                    os.chdir('/content/Teo-Moto-Repo')
                    print("✅ Repository cloned successfully")
                else:
                    print("❌ Clone failed. Please check your internet connection.")
                    return False
            except Exception as e:
                print(f"❌ Error: {e}")
                return False
        
        # Install dependencies
        print("📦 Installing dependencies...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', 'requirements.txt'], 
                      capture_output=True)
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-e', '.'], 
                      capture_output=True)
        print("✅ Dependencies installed")
    
    # Verify installation
    try:
        print("🔍 Verifying installation...")
        import pandas as pd
        import numpy as np
        import matplotlib.pyplot as plt
        import ipywidgets as widgets
        from IPython.display import display, clear_output, HTML
        import json
        import glob
        from datetime import datetime
        
        # Import project modules
        from ithaka_powertrain_sim.component_library import (
            get_motorcycle_list, get_motorcycle_specs, create_motorcycle,
            ComponentPickerState, PREDEFINED_MOTORCYCLES
        )
        from ithaka_powertrain_sim.motorbike import Motorbike
        from ithaka_powertrain_sim.trajectory import load_gpx, append_and_resample_dataframe
        
        print("✅ All modules imported successfully")
        print(f"📊 Found {len(get_motorcycle_list())} predefined motorcycles")
        
        gpx_files = glob.glob('docs/gpx_files/*.gpx')
        if gpx_files:
            print(f"🗺️ Found {len(gpx_files)} test tracks")
        
        return True
        
    except ImportError as e:
        print(f"❌ Import failed: {e}")
        if 'shortuuid' in str(e):
            print("💡 Installing missing dependency...")
            subprocess.run([sys.executable, '-m', 'pip', 'install', 'shortuuid'], 
                         capture_output=True)
            print("✅ Please re-run this cell")
        return False

# Run setup
print("🚀 Starting environment setup...")
setup_success = setup_environment()

if setup_success:
    print("\n🎉 Setup complete! Ready to design motorcycles!")
    print("👇 Continue to the next section")
else:
    print("\n❌ Setup failed. Please check the error messages above.")

In [ ]:
    def simulate_motorcycle_on_track(self, motorcycle, track_file, track_name):
        """Run simulation for motorcycle on single track."""
        try:
            # Load trajectory
            trajectory_df = load_gpx(track_file)
            trajectory_df = append_and_resample_dataframe(trajectory_df)
            
            # Extract parameters
            target_speed = trajectory_df["Target Speed"].to_list()
            delta_distance = np.diff(trajectory_df["Distance"], prepend=0).tolist()
            delta_elevation = np.diff(trajectory_df["Elevation"], prepend=0).tolist()
            approximate_time = trajectory_df["Target Time"].to_list()
            
            # Run simulation
            achieved_speeds = [trajectory_df["Target Speed"].iloc[0]]
            reporting_dataframe_rows = []
            
            for index in range(1, len(trajectory_df)):
                delta_time = approximate_time[index] - approximate_time[index - 1]
                
                achieved_speed, reporting_dataframe_row = motorcycle.calculate_achieved_speed(
                    achieved_speeds[index - 1], target_speed[index], delta_time,
                    delta_distance[index], delta_elevation[index]
                )
                
                achieved_speeds.append(achieved_speed)
                reporting_dataframe_rows.append(reporting_dataframe_row)
                
                # Adjust timing
                if index < len(trajectory_df) - 1 and achieved_speed != 0 and target_speed[index] != 0:
                    future_delta_time = (delta_distance[index + 1] / achieved_speed) - (
                        delta_distance[index + 1] / target_speed[index]
                    )
                    for j in range(index + 1, len(approximate_time)):
                        approximate_time[j] += future_delta_time
            
            # Combine results - Fixed to handle potential duplicate index issues
            motorbike_dataframe = trajectory_df.copy()
            motorbike_dataframe["Approximate Time"] = approximate_time
            motorbike_dataframe["Achieved Speed"] = achieved_speeds
            
            if reporting_dataframe_rows:
                reporting_dataframe = pd.concat(reporting_dataframe_rows, ignore_index=True)
                # Use merge instead of join to avoid duplicate index issues
                # Reset index on both dataframes to ensure clean merge
                motorbike_dataframe = motorbike_dataframe.reset_index(drop=True)
                reporting_dataframe = reporting_dataframe.reset_index(drop=True)
                # Concatenate along columns (axis=1) instead of join
                final_results = pd.concat([motorbike_dataframe, reporting_dataframe], axis=1)
            else:
                final_results = motorbike_dataframe
            
            # Calculate metrics
            total_time = approximate_time[-1] if approximate_time else 0
            total_distance = trajectory_df['Distance'].iloc[-1] if len(trajectory_df) > 0 else 0
            avg_speed = (total_distance / total_time * 3.6) if total_time > 0 else 0
            
            energy_consumed = 0
            if reporting_dataframe_rows and 'Energy Consumed (J)' in reporting_dataframe.columns:
                energy_consumed = reporting_dataframe['Energy Consumed (J)'].sum()
            
            return {
                'track_name': track_name,
                'track_file': track_file,
                'total_time_s': total_time,
                'total_distance_km': total_distance / 1000,
                'average_speed_kmh': avg_speed,
                'energy_consumed_kWh': energy_consumed / 3.6e6,
                'simulation_results': final_results,
                'trajectory': trajectory_df,
                'success': True
            }
            
        except Exception as e:
            return {
                'track_name': track_name,
                'track_file': track_file,
                'error': str(e),
                'success': False
            }

In [ ]:
## Step 1: Choose Your Design Approach

Select whether you want to use a predefined motorcycle or build a custom configuration:

In [ ]:
#@title 🎯 Select Design Method { display-mode: "form" }

#@markdown Choose how you want to create your motorcycle:
#@markdown - **Predefined Motorcycles**: Select from our library of professionally configured motorcycles
#@markdown - **Custom Motorcycle**: Build your own configuration with custom components

workflow_selector = widgets.RadioButtons(
    options=[
        ('🏍️ Choose from Predefined Motorcycles', 'predefined'),
        ('🔧 Build Custom Motorcycle', 'custom')
    ],
    value=None,
    description='Design Method:',
    style=style,
    layout=layout_wide
)

workflow_output = widgets.Output()

def handle_workflow_selection(change):
    app.workflow_choice = change.new
    with workflow_output:
        clear_output()
        if change.new == 'predefined':
            print("✅ Selected: Predefined Motorcycles")
            print("👇 Continue to Step 2 to choose a motorcycle")
        elif change.new == 'custom':
            print("✅ Selected: Custom Motorcycle Builder")
            print("👇 Continue to Step 2 to configure components")

workflow_selector.observe(handle_workflow_selection, names='value')

display(workflow_selector)
display(workflow_output)

In [ ]:
#@title 🏍️ Step 2: Motorcycle Configuration Interface { display-mode: "form" }

#@markdown This creates the interface for selecting motorcycles or building custom configurations.
#@markdown **The interface will appear below** after you run this cell.

from ithaka_powertrain_sim.component_library import (
    get_motorcycle_list, get_motorcycle_specs, create_motorcycle,
    ComponentPickerState, PREDEFINED_MOTORCYCLES
)
from datetime import datetime

# Create all UI elements
motorcycle_selector = widgets.Dropdown(
    options=[("Select a motorcycle...", None)],
    value=None,
    description='Motorcycle:',
    style=style,
    layout=layout_wide
)

motorcycle_info = widgets.Output()
confirm_selection_btn = widgets.Button(
    description='✅ Select This Motorcycle',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# Custom motorcycle interface
custom_info = widgets.Output()
custom_interface = widgets.Output()
build_custom_btn = widgets.Button(
    description='🔧 Build Custom Motorcycle',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

# Update motorcycle dropdown when workflow is selected
def update_motorcycle_options():
    if app.workflow_choice == 'predefined':
        try:
            options = [("Select a motorcycle...", None)]
            for name in get_motorcycle_list():
                specs = get_motorcycle_specs(name)
                motorcycle_type = specs.get('type', 'Unknown')
                options.append((f"{name} ({motorcycle_type})", name))
            motorcycle_selector.options = options
        except Exception as e:
            print(f"Error loading motorcycles: {e}")

def display_motorcycle_info(change):
    if change.new is None or app.workflow_choice != 'predefined':
        return
    
    with motorcycle_info:
        clear_output()
        try:
            specs = get_motorcycle_specs(change.new)
            bike = create_motorcycle(change.new)
            
            print(f"📋 {change.new} Specifications")
            print("=" * 40)
            print(f"Type: {specs.get('type', 'Unknown')}")
            print(f"Total Mass: {bike.mass:.1f} kg")
            
            # Calculate total power
            total_power = 0
            for component in bike.child_components:
                if hasattr(component, '_maximum_power_generation'):
                    total_power += component._maximum_power_generation
            
            if total_power > 0:
                print(f"Max Power: {total_power / 1000:.1f} kW")
                print(f"Power-to-Weight: {(total_power / 1000) / bike.mass:.3f} kW/kg")
            
            if 'battery_kWh' in specs:
                print(f"Battery: {specs['battery_kWh']} kWh")
                if 'range_km' in specs:
                    print(f"Range: {specs['range_km']} km")
            
            if 'displacement_cc' in specs:
                print(f"Engine: {specs['displacement_cc']} cc")
                if 'fuel_L' in specs:
                    print(f"Fuel: {specs['fuel_L']} L")
            
            print(f"\nDescription: {specs.get('description', 'No description available')}")
            
        except Exception as e:
            print(f"Error loading motorcycle info: {e}")

def confirm_motorcycle_selection(btn):
    if motorcycle_selector.value and app.workflow_choice == 'predefined':
        try:
            app.selected_motorcycle = create_motorcycle(motorcycle_selector.value)
            with motorcycle_info:
                clear_output()
                print(f"✅ {motorcycle_selector.value} selected successfully!")
                print("👇 Continue to Step 3 to choose test tracks")
        except Exception as e:
            with motorcycle_info:
                clear_output()
                print(f"❌ Error selecting motorcycle: {e}")

def setup_custom_interface(btn):
    if app.workflow_choice == 'custom':
        with custom_interface:
            clear_output()
            try:
                app.picker_state = ComponentPickerState()
                app.picker_state.display_interface()
                
                # Add build button after interface
                def build_custom_motorcycle(build_btn):
                    with custom_info:
                        clear_output()
                        try:
                            # Build from picker state
                            components = []
                            
                            # Add selected components
                            if hasattr(app.picker_state, 'selected_engine') and app.picker_state.selected_engine:
                                components.append(app.picker_state.selected_engine)
                            if hasattr(app.picker_state, 'selected_motor') and app.picker_state.selected_motor:
                                components.append(app.picker_state.selected_motor)
                            if hasattr(app.picker_state, 'selected_battery') and app.picker_state.selected_battery:
                                components.append(app.picker_state.selected_battery)
                            if hasattr(app.picker_state, 'selected_fuel_tank') and app.picker_state.selected_fuel_tank:
                                components.append(app.picker_state.selected_fuel_tank)
                                
                            if not components:
                                print("❌ No components selected. Please select at least one component.")
                                return
                            
                            # Create custom motorcycle
                            from ithaka_powertrain_sim.motorbike import Motorbike
                            app.custom_motorcycle = Motorbike(
                                name=f"Custom_{datetime.now().strftime('%H%M%S')}",
                                child_components=components,
                                dry_mass_excluding_components=150.0,  # Base motorcycle mass
                                front_mass_ratio=0.4,
                                wheelbase=1.4,
                                front_wheel_radius=0.3,
                                rear_wheel_radius=0.3,
                                frontal_area=0.6,
                                coefficient_of_aerodynamic_drag=0.6
                            )
                            
                            print(f"✅ Custom motorcycle built successfully!")
                            print(f"Total Mass: {app.custom_motorcycle.mass:.1f} kg")
                            print(f"Components: {len(components)}")
                            print("👇 Continue to Step 3 to choose test tracks")
                            
                        except Exception as e:
                            print(f"❌ Error building motorcycle: {e}")
                
                final_build_btn = widgets.Button(
                    description='🏗️ Build My Motorcycle',
                    button_style='success',
                    layout=widgets.Layout(width='200px')
                )
                final_build_btn.on_click(build_custom_motorcycle)
                display(final_build_btn)
                
            except Exception as e:
                print(f"Error setting up custom interface: {e}")

# Connect event handlers
motorcycle_selector.observe(display_motorcycle_info, names='value')
confirm_selection_btn.on_click(confirm_motorcycle_selection)
build_custom_btn.on_click(setup_custom_interface)

# Display appropriate interface based on workflow choice
def show_motorcycle_interface():
    if app.workflow_choice == 'predefined':
        update_motorcycle_options()
        display(motorcycle_selector)
        display(motorcycle_info)
        display(confirm_selection_btn)
        print("\n" + "="*50)
        
    elif app.workflow_choice == 'custom':
        display(build_custom_btn)
        display(custom_interface)
        display(custom_info)
        print("\n" + "="*50)
    else:
        print("⚠️ Please select a design method in Step 1 first")

# Show interface if workflow is already selected
if hasattr(app, 'workflow_choice') and app.workflow_choice:
    show_motorcycle_interface()
else:
    print("⚠️ Please complete Step 1 first")

In [ ]:
#@title 🏍️ Configure Your Motorcycle { display-mode: "form" }

#@markdown This section provides interfaces for either selecting a predefined motorcycle or building a custom one.
#@markdown **Run this cell** and the appropriate interface will appear based on your choice in Step 1.

# Import component builders
from ithaka_powertrain_sim.component_library import (
    Battery_10kWh_200WhKg, Battery_15kWh_220WhKg, Battery_20kWh_180WhKg, Battery_25kWh_200WhKg,
    Motor_30kW_MidDrive, Motor_50kW_HighPerf, Motor_80kW_HighPerf, Motor_120kW_HighPerf,
    Engine_400cc_30kW, Engine_500cc_40kW, Engine_650cc_50kW, Engine_750cc_60kW, Engine_1000cc_80kW,
    FuelTank_15L, FuelTank_25L
)

# Predefined motorcycle interface components
motorcycle_selector = widgets.Dropdown(
    options=[("Select a motorcycle...", None)],
    value=None,
    description='Motorcycle:',
    style=style,
    layout=layout_wide
)

motorcycle_info = widgets.Output()
confirm_selection_btn = widgets.Button(
    description='✅ Select This Motorcycle',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# Custom motorcycle interface components
custom_interface = widgets.Output()
custom_info = widgets.Output()

def update_motorcycle_options():
    """Update dropdown with available motorcycles."""
    if app.workflow_choice == 'predefined':
        try:
            options = [("Select a motorcycle...", None)]
            for name in get_motorcycle_list():
                specs = get_motorcycle_specs(name)
                motorcycle_type = specs.get('type', 'Unknown')
                options.append((f"{name} ({motorcycle_type})", name))
            motorcycle_selector.options = options
        except Exception as e:
            print(f"Error loading motorcycles: {e}")

def display_motorcycle_info(change):
    """Display detailed information about selected motorcycle."""
    if change.new is None or app.workflow_choice != 'predefined':
        return
    
    with motorcycle_info:
        clear_output()
        try:
            specs = get_motorcycle_specs(change.new)
            bike = create_motorcycle(change.new)
            
            print(f"📋 {change.new} Specifications")
            print("=" * 40)
            print(f"Type: {specs.get('type', 'Unknown')}")
            print(f"Total Mass: {bike.mass:.1f} kg")
            
            # Calculate total power
            total_power = 0
            for component in bike.child_components:
                if hasattr(component, '_maximum_power_generation'):
                    total_power += component._maximum_power_generation
            
            if total_power > 0:
                print(f"Max Power: {total_power / 1000:.1f} kW")
                print(f"Power-to-Weight: {(total_power / 1000) / bike.mass:.3f} kW/kg")
            
            if 'battery_kWh' in specs:
                print(f"Battery: {specs['battery_kWh']} kWh")
            
            if 'displacement_cc' in specs:
                print(f"Engine: {specs['displacement_cc']} cc")
            
            if 'fuel_L' in specs:
                print(f"Fuel Tank: {specs['fuel_L']} L")
            
            print(f"\nDescription: {specs.get('description', 'No description available')}")
            
        except Exception as e:
            print(f"Error loading motorcycle info: {e}")

def confirm_motorcycle_selection(btn):
    """Confirm selection of predefined motorcycle."""
    if motorcycle_selector.value and app.workflow_choice == 'predefined':
        try:
            app.selected_motorcycle = create_motorcycle(motorcycle_selector.value)
            with motorcycle_info:
                clear_output()
                print(f"✅ {motorcycle_selector.value} selected successfully!")
                print("👇 Continue to Step 3 to choose test tracks")
        except Exception as e:
            with motorcycle_info:
                clear_output()
                print(f"❌ Error selecting motorcycle: {e}")

def create_custom_interface():
    """Create the custom motorcycle builder interface."""
    with custom_interface:
        clear_output()
        
        print("🔧 Custom Motorcycle Builder")
        print("=" * 40)
        print("Build your motorcycle by selecting components below:")
        print()
        
        # Motorcycle type selector
        type_selector = widgets.RadioButtons(
            options=['Pure EV', 'Hybrid', 'Pure ICE'],
            value='Pure EV',
            description='Type:',
            style=style
        )
        
        # Component selectors
        motor_selector = widgets.Dropdown(
            options=[
                ("Select motor...", None),
                ("30kW Mid-Drive Motor", Motor_30kW_MidDrive),
                ("50kW High-Performance Motor", Motor_50kW_HighPerf),
                ("80kW High-Performance Motor", Motor_80kW_HighPerf),
                ("120kW High-Performance Motor", Motor_120kW_HighPerf)
            ],
            value=None,
            description='Electric Motor:',
            style=style,
            layout=layout_wide
        )
        
        battery_selector = widgets.Dropdown(
            options=[
                ("Select battery...", None),
                ("10kWh Battery (200 Wh/kg)", Battery_10kWh_200WhKg),
                ("15kWh Battery (220 Wh/kg)", Battery_15kWh_220WhKg),
                ("20kWh Battery (180 Wh/kg)", Battery_20kWh_180WhKg),
                ("25kWh Battery (200 Wh/kg)", Battery_25kWh_200WhKg)
            ],
            value=None,
            description='Battery:',
            style=style,
            layout=layout_wide
        )
        
        engine_selector = widgets.Dropdown(
            options=[
                ("Select engine...", None),
                ("400cc Engine (30kW)", Engine_400cc_30kW),
                ("500cc Engine (40kW)", Engine_500cc_40kW),
                ("650cc Engine (50kW)", Engine_650cc_50kW),
                ("750cc Engine (60kW)", Engine_750cc_60kW),
                ("1000cc Engine (80kW)", Engine_1000cc_80kW)
            ],
            value=None,
            description='ICE Engine:',
            style=style,
            layout=layout_wide
        )
        
        fuel_tank_selector = widgets.Dropdown(
            options=[
                ("Select fuel tank...", None),
                ("15L Fuel Tank", FuelTank_15L),
                ("25L Fuel Tank", FuelTank_25L)
            ],
            value=None,
            description='Fuel Tank:',
            style=style,
            layout=layout_wide
        )
        
        # Component containers
        motor_container = widgets.VBox([motor_selector])
        battery_container = widgets.VBox([battery_selector])
        engine_container = widgets.VBox([engine_selector])
        fuel_container = widgets.VBox([fuel_tank_selector])
        
        # Metrics display
        metrics_display = widgets.Output()
        
        def update_component_visibility(change=None):
            """Update which components are visible based on motorcycle type."""
            moto_type = type_selector.value
            
            if moto_type == "Pure EV":
                motor_container.layout.display = 'block'
                battery_container.layout.display = 'block'
                engine_container.layout.display = 'none'
                fuel_container.layout.display = 'none'
            elif moto_type == "Pure ICE":
                motor_container.layout.display = 'none'
                battery_container.layout.display = 'none'
                engine_container.layout.display = 'block'
                fuel_container.layout.display = 'block'
            elif moto_type == "Hybrid":
                motor_container.layout.display = 'block'
                battery_container.layout.display = 'block'
                engine_container.layout.display = 'block'
                fuel_container.layout.display = 'block'
        
        def update_metrics(change=None):
            """Update configuration metrics."""
            with metrics_display:
                clear_output()
                
                components = []
                total_mass = 150.0  # Base motorcycle mass
                total_power = 0
                
                # Collect selected components
                if motor_selector.value and motor_container.layout.display != 'none':
                    comp = motor_selector.value()
                    components.append(comp)
                    total_mass += comp.mass
                    if hasattr(comp, '_maximum_power_generation'):
                        total_power += comp._maximum_power_generation
                
                if battery_selector.value and battery_container.layout.display != 'none':
                    comp = battery_selector.value()
                    components.append(comp)
                    total_mass += comp.mass
                
                if engine_selector.value and engine_container.layout.display != 'none':
                    comp = engine_selector.value()
                    components.append(comp)
                    total_mass += comp.mass
                    if hasattr(comp, '_maximum_power_generation'):
                        total_power += comp._maximum_power_generation
                
                if fuel_tank_selector.value and fuel_container.layout.display != 'none':
                    comp = fuel_tank_selector.value()
                    components.append(comp)
                    total_mass += comp.mass
                
                # Display metrics
                print(f"🏍️ Type: {type_selector.value}")
                print(f"⚖️ Total Weight: {total_mass:.1f} kg")
                print(f"🔧 Components: {len(components)}")
                
                if total_power > 0:
                    power_to_weight = (total_power / 1000) / total_mass
                    print(f"⚡ Total Power: {total_power / 1000:.1f} kW")
                    print(f"🏁 Power-to-Weight: {power_to_weight:.3f} kW/kg")
        
        # Connect event handlers
        type_selector.observe(update_component_visibility, names='value')
        type_selector.observe(update_metrics, names='value')
        motor_selector.observe(update_metrics, names='value')
        battery_selector.observe(update_metrics, names='value')
        engine_selector.observe(update_metrics, names='value')
        fuel_tank_selector.observe(update_metrics, names='value')
        
        # Initial setup
        update_component_visibility()
        update_metrics()
        
        # Display interface
        display(HTML("<h4>1️⃣ Select Motorcycle Type</h4>"))
        display(type_selector)
        
        display(HTML("<h4>2️⃣ Select Components</h4>"))
        display(motor_container)
        display(battery_container)
        display(engine_container)
        display(fuel_container)
        
        display(HTML("<h4>3️⃣ Configuration Summary</h4>"))
        display(metrics_display)
        
        # Build button
        display(HTML("<h4>4️⃣ Build Your Motorcycle</h4>"))
        
        def build_custom_motorcycle(btn):
            with custom_info:
                clear_output()
                try:
                    # Collect components
                    components = []
                    
                    if motor_selector.value and motor_container.layout.display != 'none':
                        components.append(motor_selector.value())
                    
                    if battery_selector.value and battery_container.layout.display != 'none':
                        components.append(battery_selector.value())
                    
                    if engine_selector.value and engine_container.layout.display != 'none':
                        components.append(engine_selector.value())
                    
                    if fuel_tank_selector.value and fuel_container.layout.display != 'none':
                        components.append(fuel_tank_selector.value())
                    
                    if not components:
                        print("❌ No components selected. Please select at least one component.")
                        return
                    
                    # Create custom motorcycle
                    app.custom_motorcycle = Motorbike(
                        name=f"Custom_{type_selector.value.replace(' ', '_')}_{datetime.now().strftime('%H%M%S')}",
                        child_components=components,
                        dry_mass_excluding_components=150.0,
                        front_mass_ratio=0.4,
                        wheelbase=1.4,
                        front_wheel_radius=0.3,
                        rear_wheel_radius=0.3,
                        frontal_area=0.6,
                        coefficient_of_aerodynamic_drag=0.6
                    )
                    
                    print(f"✅ Custom motorcycle built successfully!")
                    print(f"Type: {type_selector.value}")
                    print(f"Total Mass: {app.custom_motorcycle.mass:.1f} kg")
                    print(f"Components: {len(components)}")
                    
                    # Show component list
                    print("\nComponents included:")
                    for comp in components:
                        print(f"  • {comp.name}")
                    
                    print("\n👇 Continue to Step 3 to choose test tracks")
                    
                except Exception as e:
                    print(f"❌ Error building motorcycle: {e}")
        
        build_btn = widgets.Button(
            description='🏗️ Build My Motorcycle',
            button_style='success',
            layout=widgets.Layout(width='200px')
        )
        build_btn.on_click(build_custom_motorcycle)
        display(build_btn)

# Connect event handlers
motorcycle_selector.observe(display_motorcycle_info, names='value')
confirm_selection_btn.on_click(confirm_motorcycle_selection)

# Display appropriate interface based on workflow choice
if app.workflow_choice == 'predefined':
    update_motorcycle_options()
    display(motorcycle_selector)
    display(motorcycle_info)
    display(confirm_selection_btn)
    
elif app.workflow_choice == 'custom':
    create_custom_interface()
    display(custom_info)
    
else:
    print("⚠️ Please select a design method in Step 1 first")

In [ ]:
## Step 3: Select Test Tracks

Choose which GPS tracks to test your motorcycle on:

In [ ]:
#@title 🗺️ Track Selection { display-mode: "form" }

#@markdown Select which tracks to test your motorcycle on:
#@markdown - **Single Track**: Test on one specific track
#@markdown - **All Tracks**: Test on all available tracks for comprehensive analysis

track_mode_selector = widgets.RadioButtons(
    options=[
        ('🎯 Test Single Track', 'single'),
        ('🌍 Test All Available Tracks', 'all')
    ],
    value=None,
    description='Test Mode:',
    style=style,
    layout=layout_wide
)

single_track_selector = widgets.Dropdown(
    options=[("Select a track...", None)],
    value=None,
    description='Track:',
    style=style,
    layout=layout_wide
)

track_info = widgets.Output()
confirm_tracks_btn = widgets.Button(
    description='✅ Confirm Track Selection',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

def update_track_options():
    """Update track dropdown with available tracks."""
    try:
        available_tracks = app.get_available_tracks()
        if available_tracks and available_tracks[0][1] is not None:
            options = [("Select a track...", None)] + available_tracks
            single_track_selector.options = options
        else:
            single_track_selector.options = [("No tracks available", None)]
    except Exception as e:
        print(f"Error loading tracks: {e}")

def handle_track_mode_change(change):
    """Handle track mode selection."""
    with track_info:
        clear_output()
        if change.new == 'single':
            update_track_options()
            single_track_selector.layout.display = 'block'
            print("📍 Select a single track to test on")
        elif change.new == 'all':
            single_track_selector.layout.display = 'none'
            available_tracks = app.get_available_tracks()
            if available_tracks and available_tracks[0][1] is not None:
                print(f"🌍 Will test on all {len(available_tracks)} available tracks:")
                for name, _ in available_tracks[:5]:
                    print(f"  • {name}")
                if len(available_tracks) > 5:
                    print(f"  ... and {len(available_tracks) - 5} more")
            else:
                print("❌ No tracks found")

def handle_single_track_change(change):
    """Handle single track selection."""
    if change.new and track_mode_selector.value == 'single':
        with track_info:
            clear_output()
            track_name = change.new[0] if isinstance(change.new, tuple) else str(change.new)
            print(f"📍 Selected: {track_name}")

def confirm_track_selection(btn):
    """Confirm track selection."""
    current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
    if not current_motorcycle:
        with track_info:
            clear_output()
            print("❌ Please select a motorcycle first (Step 2)")
        return
    
    if not track_mode_selector.value:
        with track_info:
            clear_output()
            print("❌ Please select a test mode")
        return
    
    if track_mode_selector.value == 'single' and not single_track_selector.value:
        with track_info:
            clear_output()
            print("❌ Please select a track")
        return
    
    # Store selection
    if track_mode_selector.value == 'single':
        app.selected_tracks = [single_track_selector.value]
    else:
        available_tracks = app.get_available_tracks()
        app.selected_tracks = [(name, path) for name, path in available_tracks if path is not None]
    
    with track_info:
        clear_output()
        if track_mode_selector.value == 'single':
            track_name = app.selected_tracks[0][0] if isinstance(app.selected_tracks[0], tuple) else str(app.selected_tracks[0])
            print(f"✅ Track selected: {track_name}")
        else:
            print(f"✅ All {len(app.selected_tracks)} tracks selected")
        print("👇 Continue to Step 4 to run simulation")

# Connect event handlers
track_mode_selector.observe(handle_track_mode_change, names='value')
single_track_selector.observe(handle_single_track_change, names='value')
confirm_tracks_btn.on_click(confirm_track_selection)

# Hide single track selector initially
single_track_selector.layout.display = 'none'

# Check if we have a motorcycle selected
current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
if current_motorcycle:
    display(track_mode_selector)
    display(single_track_selector)
    display(track_info)
    display(confirm_tracks_btn)
else:
    print("⚠️ Please complete Step 2 (select/build a motorcycle) first")

In [ ]:
## Step 4: Run Simulation

Execute the physics-based powertrain simulation:

In [ ]:
#@title 🚀 Run Simulation { display-mode: "form" }

#@markdown Click the button below to start the simulation.
#@markdown The simulation will calculate:
#@markdown - Achieved speeds vs target speeds
#@markdown - Energy consumption
#@markdown - Performance metrics
#@markdown - Component utilization

run_simulation_btn = widgets.Button(
    description='🚀 Run Simulation',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

simulation_progress = widgets.Output()

def run_simulation(btn):
    """Execute simulation for selected motorcycle and tracks."""
    # Check prerequisites
    current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
    if not current_motorcycle:
        with simulation_progress:
            clear_output()
            print("❌ No motorcycle selected. Complete Step 2 first.")
        return
    
    if not app.selected_tracks:
        with simulation_progress:
            clear_output()
            print("❌ No tracks selected. Complete Step 3 first.")
        return
    
    with simulation_progress:
        clear_output()
        print("🚀 Starting simulation...")
        print(f"🏍️ Motorcycle: {current_motorcycle.name}")
        print(f"📊 Tracks to test: {len(app.selected_tracks)}")
        print("=" * 50)
        
        app.simulation_results = {}
        successful_runs = 0
        
        for i, (track_name, track_file) in enumerate(app.selected_tracks):
            if track_file is None:
                continue
                
            print(f"\n📍 ({i+1}/{len(app.selected_tracks)}) Testing: {track_name}")
            
            try:
                result = app.simulate_motorcycle_on_track(current_motorcycle, track_file, track_name)
                app.simulation_results[track_name] = result
                
                if result['success']:
                    successful_runs += 1
                    print(f"  ✅ Completed in {result['total_time_s']/60:.1f} min")
                    print(f"     Distance: {result['total_distance_km']:.1f} km")
                    print(f"     Avg Speed: {result['average_speed_kmh']:.1f} km/h")
                    if result['energy_consumed_kWh'] > 0:
                        print(f"     Energy Used: {result['energy_consumed_kWh']:.2f} kWh")
                else:
                    print(f"  ❌ Failed: {result.get('error', 'Unknown error')}")
            
            except Exception as e:
                print(f"  ❌ Exception: {str(e)}")
                app.simulation_results[track_name] = {
                    'track_name': track_name,
                    'error': str(e),
                    'success': False
                }
        
        print("\n" + "=" * 50)
        print(f"🎉 Simulation Complete!")
        print(f"✅ Successful: {successful_runs}/{len(app.selected_tracks)} tracks")
        
        if successful_runs > 0:
            print("\n👇 Continue to Step 5 to view detailed results and export data")
        else:
            print("\n❌ No successful simulations. Check track files and motorcycle configuration.")

run_simulation_btn.on_click(run_simulation)

# Only show if prerequisites are met
current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
if current_motorcycle and app.selected_tracks:
    display(run_simulation_btn)
    display(simulation_progress)
else:
    missing = []
    if not current_motorcycle:
        missing.append("motorcycle (Step 2)")
    if not app.selected_tracks:
        missing.append("tracks (Step 3)")
    
    print(f"⚠️ Complete the following first: {', '.join(missing)}")

In [ ]:
## Step 5: Results Analysis & Export

View comprehensive simulation results and export data for further analysis:

In [ ]:
#@title 📊 Results Analysis & Export { display-mode: "form" }

#@markdown This section provides detailed analysis of simulation results and export options.
#@markdown **Use the buttons below** to view results and export comprehensive data.

view_results_btn = widgets.Button(
    description='📊 View Results',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

export_results_btn = widgets.Button(
    description='💾 Export JSON',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

results_display = widgets.Output()
export_status = widgets.Output()

def view_detailed_results(btn):
    """Display comprehensive simulation results."""
    if not app.simulation_results:
        with results_display:
            clear_output()
            print("❌ No simulation results available. Run simulation first.")
        return
    
    with results_display:
        clear_output()
        
        # Get motorcycle info
        current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
        
        print("📊 SIMULATION RESULTS SUMMARY")
        print("=" * 60)
        
        if current_motorcycle:
            print(f"🏍️ Motorcycle: {current_motorcycle.name}")
            print(f"⚖️ Total Mass: {current_motorcycle.mass:.1f} kg")
            
            # Calculate total power
            total_power = 0
            for component in current_motorcycle.child_components:
                if hasattr(component, '_maximum_power_generation'):
                    total_power += component._maximum_power_generation
            
            if total_power > 0:
                power_to_weight = (total_power / 1000) / current_motorcycle.mass
                print(f"⚡ Total Power: {total_power / 1000:.1f} kW")
                print(f"🏁 Power-to-Weight Ratio: {power_to_weight:.3f} kW/kg")
            print()
        
        # Results table
        print("📈 TRACK PERFORMANCE RESULTS:")
        print("-" * 85)
        print(f"{'Track Name':<30} {'Distance':<10} {'Time':<8} {'Avg Speed':<12} {'Energy':<10}")
        print(f"{'':30} {'(km)':<10} {'(min)':<8} {'(km/h)':<12} {'(kWh)':<10}")
        print("-" * 85)
        
        successful_results = []
        for track_name, result in app.simulation_results.items():
            if result.get('success'):
                dist = result['total_distance_km']
                time_min = result['total_time_s'] / 60
                speed = result['average_speed_kmh']
                energy = result['energy_consumed_kWh']
                print(f"{track_name:<30} {dist:<10.1f} {time_min:<8.1f} {speed:<12.1f} {energy:<10.2f}")
                successful_results.append(result)
            else:
                error_msg = result.get('error', 'Unknown error')[:15] + "..."
                print(f"{track_name:<30} {'ERROR':<10} {'---':<8} {'---':<12} {'---':<10}")
        
        print("-" * 85)
        
        # Summary statistics
        if successful_results:
            avg_speed = np.mean([r['average_speed_kmh'] for r in successful_results])
            total_distance = sum([r['total_distance_km'] for r in successful_results])
            total_energy = sum([r['energy_consumed_kWh'] for r in successful_results])
            total_time = sum([r['total_time_s'] for r in successful_results]) / 3600  # hours
            
            print(f"\n📊 SUMMARY STATISTICS:")
            print(f"✅ Successful simulations: {len(successful_results)}/{len(app.simulation_results)}")
            print(f"🏁 Average speed across all tracks: {avg_speed:.1f} km/h")
            print(f"🛣️ Total distance simulated: {total_distance:.1f} km")
            print(f"⏱️ Total simulation time: {total_time:.1f} hours")
            print(f"🔋 Total energy consumed: {total_energy:.2f} kWh")
            
            if total_distance > 0:
                efficiency = total_energy / total_distance
                print(f"⚡ Average energy efficiency: {efficiency:.3f} kWh/km")
                
                if total_energy > 0:
                    range_per_kwh = total_distance / total_energy
                    print(f"📏 Range per kWh: {range_per_kwh:.1f} km/kWh")

def export_simulation_results(btn):
    """Export comprehensive simulation results to JSON."""
    if not app.simulation_results:
        with export_status:
            clear_output()
            print("❌ No results to export. Run simulation first.")
        return
    
    current_motorcycle = app.selected_motorcycle or app.custom_motorcycle
    if not current_motorcycle:
        with export_status:
            clear_output()
            print("❌ No motorcycle data available.")
        return
    
    try:
        with export_status:
            clear_output()
            print("💾 Preparing comprehensive export...")
        
        # Create comprehensive export data
        export_data = {
            'export_metadata': {
                'timestamp': datetime.now().isoformat(),
                'workflow_type': app.workflow_choice,
                'application_version': '2.0',
                'total_tracks_tested': len(app.simulation_results)
            },
            'motorcycle_specifications': {
                'name': current_motorcycle.name,
                'total_mass_kg': float(current_motorcycle.mass),
                'dry_mass_excluding_components_kg': float(current_motorcycle.dry_mass_excluding_components),
                'front_mass_ratio': float(current_motorcycle.front_mass_ratio),
                'front_wheel_radius_m': float(current_motorcycle.front_wheel_radius),
                'rear_wheel_radius_m': float(current_motorcycle.rear_wheel_radius),
                'frontal_area_m2': float(current_motorcycle.frontal_area),
                'coefficient_of_aerodynamic_drag': float(current_motorcycle.coefficient_of_aerodynamic_drag),
                'components': []
            },
            'simulation_results': {},
            'performance_summary': {}
        }
        
        # Add component specifications
        total_power = 0
        for component in current_motorcycle.child_components:
            comp_spec = {
                'name': component.name,
                'type': type(component).__name__,
                'mass_kg': float(component.mass)
            }
            
            if hasattr(component, '_maximum_power_generation'):
                power_w = component._maximum_power_generation
                comp_spec['max_power_W'] = float(power_w)
                comp_spec['max_power_kW'] = float(power_w / 1000)
                total_power += power_w
            
            if hasattr(component, 'remaining_energy_capacity'):
                comp_spec['energy_capacity_kWh'] = float(component.remaining_energy_capacity / 3.6e6)
                
            export_data['motorcycle_specifications']['components'].append(comp_spec)
        
        export_data['motorcycle_specifications']['total_power_kW'] = float(total_power / 1000)
        if current_motorcycle.mass > 0:
            export_data['motorcycle_specifications']['power_to_weight_ratio'] = float((total_power / 1000) / current_motorcycle.mass)
        
        # Add simulation results
        successful_results = []
        for track_name, result in app.simulation_results.items():
            if result.get('success'):
                export_data['simulation_results'][track_name] = {
                    'track_name': result['track_name'],
                    'track_file': result['track_file'],
                    'total_time_s': float(result['total_time_s']),
                    'total_distance_km': float(result['total_distance_km']),
                    'average_speed_kmh': float(result['average_speed_kmh']),
                    'energy_consumed_kWh': float(result['energy_consumed_kWh']),
                    'status': 'success'
                }
                successful_results.append(result)
            else:
                export_data['simulation_results'][track_name] = {
                    'track_name': result['track_name'],
                    'track_file': result.get('track_file', 'unknown'),
                    'error': result.get('error', 'Unknown error'),
                    'status': 'failed'
                }
        
        # Add performance summary
        if successful_results:
            export_data['performance_summary'] = {
                'successful_simulations': len(successful_results),
                'failed_simulations': len(app.simulation_results) - len(successful_results),
                'average_speed_kmh': float(np.mean([r['average_speed_kmh'] for r in successful_results])),
                'total_distance_km': float(sum([r['total_distance_km'] for r in successful_results])),
                'total_time_hours': float(sum([r['total_time_s'] for r in successful_results]) / 3600),
                'total_energy_consumed_kWh': float(sum([r['energy_consumed_kWh'] for r in successful_results])),
            }
            
            total_dist = export_data['performance_summary']['total_distance_km']
            total_energy = export_data['performance_summary']['total_energy_consumed_kWh']
            
            if total_dist > 0 and total_energy > 0:
                export_data['performance_summary']['energy_efficiency_kWh_per_km'] = float(total_energy / total_dist)
                export_data['performance_summary']['range_per_kWh_km'] = float(total_dist / total_energy)
        
        # Generate filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        motorcycle_name = current_motorcycle.name.replace(' ', '_').replace('/', '_').lower()
        filename = f"ithaka_simulation_{motorcycle_name}_{timestamp}.json"
        
        # Write to file
        with open(filename, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        with export_status:
            clear_output()
            print("✅ Export completed successfully!")
            print(f"📄 Filename: {filename}")
            print("\n📊 Export contains:")
            print("  • Complete motorcycle specifications and component details")
            print(f"  • {len(app.simulation_results)} track simulation results")
            print("  • Performance summary and efficiency metrics")
            print("  • Metadata and timestamps for full traceability")
            
            file_size = os.path.getsize(filename) / 1024  # KB
            print(f"  • File size: {file_size:.1f} KB")
            print("\n🎯 Ready for engineering analysis and reporting!")
            
    except Exception as e:
        with export_status:
            clear_output()
            print(f"❌ Export failed: {str(e)}")

# Connect button handlers
view_results_btn.on_click(view_detailed_results)
export_results_btn.on_click(export_simulation_results)

# Only show if simulation has been run
if app.simulation_results:
    display(widgets.HBox([view_results_btn, export_results_btn]))
    display(results_display)
    display(export_status)
else:
    print("⚠️ Run simulation first (Step 4) to view and export results")

## 🎉 Workflow Complete!

### What You've Accomplished:

- **🏍️ Motorcycle Selection**: Chose from predefined motorcycles or built a custom configuration
- **🗺️ Track Testing**: Selected tracks for comprehensive performance testing
- **🚀 Simulation**: Ran physics-based powertrain simulations  
- **📊 Analysis**: Viewed detailed performance results with engineering metrics
- **💾 Export**: Generated comprehensive JSON files with all specifications and results

### Key Features:

- **🖱️ No-Code Interface**: Complete workflow using only dropdowns and buttons
- **📋 Comprehensive Data**: Full motorcycle specifications plus simulation results
- **🔧 Engineering Focus**: All data needed for powertrain analysis and optimization
- **☁️ Cloud Compatible**: Runs seamlessly in Google Colab and local Jupyter environments
- **📈 Professional Output**: Engineering-grade results and visualizations

### Next Steps:

- **🔄 Iterate Designs**: Test different motorcycle configurations
- **📊 Analyze Results**: Use exported JSON for detailed analysis in other tools
- **🏭 Scale Testing**: Run batch simulations across multiple tracks
- **📝 Document Findings**: Results are ready for technical reports and presentations

The exported JSON contains everything needed for comprehensive engineering analysis, including complete motorcycle specifications, component details, and performance metrics across all tested tracks.